In [9]:
import tensorflow as tf; 
print(tf.reduce_sum(tf.random.normal([1000, 1000])))

tf.Tensor(1117.6307, shape=(), dtype=float32)


In [10]:
!python --version
import tensorflow as tf
print(tf.__version__)
import keras

Python 3.10.11
2.15.0


AttributeError: partially initialized module 'keras.src' has no attribute 'utils' (most likely due to a circular import)

In [11]:
import numpy as np
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

# Note that Keras should only be imported after the backend
# has been configured. The backend cannot be changed once the
# package is imported.
import keras

AttributeError: partially initialized module 'keras.src' has no attribute 'utils' (most likely due to a circular import)